In [40]:
#importing the required library
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation, Dropout
from keras.activations import relu, softmax
from keras.losses import categorical_crossentropy
from keras.optimizers import sgd, adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
import matplotlib.image as mpimg
import os

In [12]:
#dimensions of our images
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2, rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [45]:
#creating the model
model = Sequential()
#single chunk of convulution
model.add(Conv2D(32, (3,3), input_shape=(224,224,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(2, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["acc"])

In [82]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 43264)           

In [49]:
#importing train data
train_gen = train_datagen.flow_from_directory("../Python/chest_xray/train", target_size=(224,224), batch_size=32)
test_gen = test_datagen.flow_from_directory("../Python/chest_xray/test", target_size=(224,224), batch_size=32)
print(train_gen.class_indices)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
{'NORMAL': 0, 'PNEUMONIA': 1}


In [50]:
#Saving the best model
checkpoint = ModelCheckpoint('../Python/checkpoint-{epoch:03d}.h5', monitor='loss', verbose=0, save_best_only=True, mode='auto')

In [51]:
model_history = model.fit_generator(train_gen, epochs=3,callbacks=[checkpoint],validation_data = test_gen)

Epoch 1/3
163/163 [==============================] - 343s 2s/step - loss: 0.3224 - acc: 0.8497 - val_loss: 0.3715 - val_acc: 0.8317
Epoch 2/3
163/163 [==============================] - 327s 2s/step - loss: 0.3185 - acc: 0.8537 - val_loss: 0.3136 - val_acc: 0.7772
Epoch 3/3
163/163 [==============================] - 309s 2s/step - loss: 0.2637 - acc: 0.8832 - val_loss: 0.4258 - val_acc: 0.8221


In [54]:
#Predicting the accuracy of the model
test_accu = model.evaluate_generator(test_gen, steps=624)
print('The testing accuracy is :',test_accu[1]*100, '%')

The testing accuracy is : 82.21035599708557 %


In [55]:
#Now loading the best model
f_model = load_model('../Python/checkpoint-003.h5')

In [86]:
#Now we are loading the validation images
predict_dir_path = "../Python/chest_xray/val/PNEUMONIA/"
onlyfiles = [f for f in os.listdir(predict_dir_path) if os.path.isfile(os.path.join(predict_dir_path, f))]
print(onlyfiles)

['person1946_bacteria_4874.jpeg', 'person1946_bacteria_4875.jpeg', 'person1947_bacteria_4876.jpeg', 'person1949_bacteria_4880.jpeg', 'person1950_bacteria_4881.jpeg', 'person1951_bacteria_4882.jpeg', 'person1952_bacteria_4883.jpeg', 'person1954_bacteria_4886.jpeg']


In [87]:
for file in onlyfiles:
    img_predict_pn = image.load_img(predict_dir_path+file,target_size=(224,224))
    img_predict_normal_final = image.img_to_array(img_predict_pn)
    img_predict_normal_final = np.expand_dims(img_predict_normal_final, axis =0)
    rslt = f_model.predict(img_predict_normal_final)
    rslt = rslt [0][0]
    if rslt == 0:
        prediction = "normal"
    else:
        prediction = "pnemonia"
    
    print(file +"   "+ prediction)


person1946_bacteria_4874.jpeg   pnemonia
person1946_bacteria_4875.jpeg   pnemonia
person1947_bacteria_4876.jpeg   pnemonia
person1949_bacteria_4880.jpeg   pnemonia
person1950_bacteria_4881.jpeg   pnemonia
person1951_bacteria_4882.jpeg   pnemonia
person1952_bacteria_4883.jpeg   pnemonia
person1954_bacteria_4886.jpeg   pnemonia
